# 使用 LoRA 和 Hugging Face 高效训练大语言模型

在本文中，我们将展示如何使用[大语言模型低秩适配（Low-Rank Adaptation of Large Language Models，LoRA）](https://arxiv.org/abs/2106.09685)技术在单 GPU 上微调 110 亿参数的 FLAN-T5 XXL 模型。在此过程中，我们会使用到 Hugging Face 的 [Transformers](https://huggingface.co/docs/transformers/index)、[Accelerate](https://huggingface.co/docs/accelerate/index) 和 [PEFT](https://github.com/huggingface/peft) 库。

通过本文，你会学到：

1. 如何搭建开发环境
2. 如何加载并准备数据集
3. 如何使用 LoRA 和 bnb（即bitsandbytes） int-8 微调 T5
4. 如何评估 LoRA FLAN-T5 并将其用于推理
5. 如何比较不同方案的性价比

### 快速入门：轻量化微调（Parameter Efficient Fine-Tuning，PEFT）

[PEFT](https://github.com/huggingface/peft) 是 Hugging Face 的一个新的开源库。使用 PEFT 库，无需微调模型的全部参数，即可高效地将预训练语言模型 (Pre-trained Language Model，PLM) 适配到各种下游应用。PEFT 目前支持以下几种方法：

- LoRA：[LORA: LOW-RANK ADAPTATION OF LARGE LANGUAGE MODELS](https://arxiv.org/pdf/2106.09685.pdf)
- Prefix Tuning：[P-Tuning v2: Prompt Tuning Can Be Comparable to Fine-tuning Universally Across Scales and Tasks](https://arxiv.org/pdf/2110.07602.pdf)
- P-Tuning：[GPT Understands, Too](https://arxiv.org/pdf/2103.10385.pdf)
- Prompt Tuning：[The Power of Scale for Parameter-Efficient Prompt Tuning](https://arxiv.org/pdf/2104.08691.pdf)

*注意：本教程是在 g5.2xlarge AWS EC2 实例上创建和运行的，该实例包含 1 个 NVIDIA A10G。*

## 1. 搭建开发环境

在本例中，我们使用 AWS 预置的 [PyTorch 深度学习 AMI](https://docs.aws.amazon.com/dlami/latest/devguide/tutorial-pytorch.html)，其已安装了正确的 CUDA 驱动程序和 PyTorch。在此基础上，我们还需要安装一些 Hugging Face 库，包括 transformers 和 datasets。运行下面的代码就可安装所有需要的包。

In [1]:
# install Hugging Face Libraries
!pip install  git+https://github.com/huggingface/peft.git
!pip install "transformers==4.27.1" "datasets==2.9.0" "accelerate==0.17.1" "evaluate==0.4.0" "bitsandbytes==0.37.1" loralib --upgrade --quiet
# install additional dependencies needed for training
!pip install rouge-score tensorboard py7zr 

  Cloning https://github.com/huggingface/peft.git to c:\users\11985\appdata\local\temp\pip-req-build-3bwf80qe
  Resolved https://github.com/huggingface/peft.git to commit 34027fe813756897767b9a6f19ae7f1c4c7b418c
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for peft: filename=peft-0.3.0.dev0-py3-none-any.whl size=50810 sha256=b4a898041e90c0419af2e2eb4da857ded2f8390f40416a90cc7311624c9c766c
  Stored in directory: C:\Users\11985\AppData\Local\Temp\pip-ephem-wheel-cache-bgzptlev\wheels\13\73\6d\ff27a3703d8bad21d7e0c24cbd9dde5d7ae78f756405707a0c
Successfully built peft


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git 'C:\Users\11985\AppData\Local\Temp\pip-req-build-3bwf80qe'


     ---------------------------------------- 6.0/6.0 MB 142.7 kB/s eta 0:00:00
     -------------------------------------- 66.4/66.4 kB 905.2 kB/s eta 0:00:00
     ---------------------------------------- 4.1/4.1 MB 2.4 MB/s eta 0:00:00
     -------------------------------------- 178.2/178.2 kB 5.4 MB/s eta 0:00:00
     ---------------------------------------- 93.9/93.9 kB 5.6 MB/s eta 0:00:00
     -------------------------------------- 781.3/781.3 kB 5.5 MB/s eta 0:00:00
     -------------------------------------- 233.6/233.6 kB 7.2 MB/s eta 0:00:00
  Using cached wheel-0.40.0-py3-none-any.whl (64 kB)
     ---------------------------------------- 1.7/1.7 MB 5.8 MB/s eta 0:00:00
     -------------------------------------- 231.8/231.8 kB 7.1 MB/s eta 0:00:00
     ---------------------------------------- 46.1/46.1 kB ? eta 0:00:00
     -------------------------------------- 365.3/365.3 kB 7.6 MB/s eta 0:00:00
     -------------------------------------- 155.3/155.3 kB 4.7 MB/s eta 0:00:0

## 2.加载并准备数据集

这里，我们使用 [samsum](https://huggingface.co/datasets/samsum) 数据集，该数据集包含大约 16k 个含摘要的聊天类对话数据。这些对话由精通英语的语言学家制作。

```python
{
  "id": "13818513",
  "summary": "Amanda baked cookies and will bring Jerry some tomorrow.",
  "dialogue": "Amanda: I baked cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"
}
```

我们使用 🤗 Datasets 库中的 *​`load_dataset()`* 方法来加载 `samsum` 数据集。

In [2]:
from datasets import load_dataset

# Load dataset from the hub
dataset = load_dataset("samsum")

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

# Train dataset size: 14732
# Test dataset size: 819

Found cached dataset samsum (D:/ProgramData/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e)


  0%|          | 0/3 [00:00<?, ?it/s]

Train dataset size: 14732
Test dataset size: 819


为了训练模型，我们要用 🤗 Transformers Tokenizer 将输入文本转换为词元 ID。如果你需要了解这一方面的知识，请移步 Hugging Face 课程的 **[第 6 章](https://huggingface.co/course/chapter6/1?fw=tf)**。

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-xxl"

# Load tokenizer of FLAN-t5-XL
tokenizer = AutoTokenizer.from_pretrained(model_id)

在开始训练之前，我们还需要对数据进行预处理。生成式文本摘要属于文本生成任务。我们将文本输入给模型，模型会输出摘要。我们需要了解输入和输出文本的长度信息，以利于我们高效地批量处理这些数据。

In [4]:
from datasets import concatenate_datasets
import numpy as np
# The maximum total input sequence length after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["dialogue"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
input_lenghts = [len(x) for x in tokenized_inputs["input_ids"]]
# take 85 percentile of max length for better utilization
max_source_length = int(np.percentile(input_lenghts, 85))
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
target_lenghts = [len(x) for x in tokenized_targets["input_ids"]]
# take 90 percentile of max length for better utilization
max_target_length = int(np.percentile(target_lenghts, 90))
print(f"Max target length: {max_target_length}")

Loading cached processed dataset at D:\ProgramData\huggingface\datasets\samsum\samsum\0.0.0\f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e\cache-03b5cc31bf151e08.arrow


Max source length: 255


  0%|          | 0/16 [00:00<?, ?ba/s]

Max target length: 50


我们将在训练前统一对数据集进行预处理并将预处理后的数据集保存到磁盘。你可以在本地机器或 CPU 上运行此步骤并将其上传到 [Hugging Face Hub](https://huggingface.co/docs/hub/datasets-overview)。

In [5]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["summarize: " + item for item in sample["dialogue"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

# save datasets to disk for later easy loading
tokenized_dataset["train"].save_to_disk("data/train")
tokenized_dataset["test"].save_to_disk("data/eval")

Loading cached processed dataset at D:\ProgramData\huggingface\datasets\samsum\samsum\0.0.0\f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e\cache-df829843b4810807.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Loading cached processed dataset at D:\ProgramData\huggingface\datasets\samsum\samsum\0.0.0\f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e\cache-c6d129a5cd66b800.arrow


Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

## 3. 使用 LoRA 和 bnb int-8 微调 T5

除了 LoRA 技术，我们还使用 [bitsanbytes LLM.int8()](https://huggingface.co/blog/hf-bitsandbytes-integration) 把冻结的 LLM 量化为 int8。这使我们能够将 FLAN-T5 XXL 所需的内存降低到约四分之一。

训练的第一步是加载模型。我们使用 [philschmid/flan-t5-xxl-sharded-fp16](https://huggingface.co/philschmid/flan-t5-xxl-sharded-fp16) 模型，它是 [google/flan-t5-xxl](https://huggingface.co/google/flan-t5-xxl) 的分片版。分片可以让我们在加载模型时不耗尽内存。

In [1]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id = "philschmid/flan-t5-xxl-sharded-fp16"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map="auto")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: d:\program files\python3.7.9\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary d:\program files\python3.7.9\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...


Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

现在，我们可以使用 `peft` 为 LoRA int-8 训练作准备了。

In [6]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# Define LoRA Config 
lora_config = LoraConfig(
 r=16, 
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)
# prepare int-8 model for training
model = prepare_model_for_int8_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# trainable params: 18874368 || all params: 11154206720 || trainable%: 0.16921300163961817

trainable params: 18874368 || all params: 11154206720 || trainable%: 0.16921300163961817


如你所见，这里我们只训练了模型参数的 0.16%！这个巨大的内存增益让我们安心地微调模型，而不用担心内存问题。

接下来需要创建一个 `DataCollat​​or`，负责对输入和标签进行填充，我们使用 🤗 Transformers 库中的`DataCollat​​orForSeq2Seq` 来完成这一环节。

In [7]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

最后一步是定义训练超参 (`TrainingArguments`)。

In [8]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

output_dir="lora-flan-t5-xxl"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
		auto_find_batch_size=True,
    learning_rate=1e-3, # higher learning rate
    num_train_epochs=5,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="no",
    report_to="tensorboard",
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

运行下面的代码，开始训练模型。请注意，对于 T5，出于收敛稳定性考量，某些层我们仍保持 `float32` 精度。

In [9]:
# train model
trainer.train()

d:\program files\python3.7.9\lib\site-packages\transformers\optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
d:\program files\python3.7.9\lib\site-packages\bitsandbytes\autograd\_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss


TrainOutput(global_step=9210, training_loss=0.99450243888797, metrics={'train_runtime': 22422.9438, 'train_samples_per_second': 3.285, 'train_steps_per_second': 0.411, 'total_flos': 1.2471170806421914e+18, 'train_loss': 0.99450243888797, 'epoch': 5.0})

训练耗时约 10 小时 36 分钟，训练 10 小时的成本约为 `13.22 美元`。相比之下，如果[在 FLAN-T5-XXL 上进行全模型微调](https://www.philschmid.de/fine-tune-flan-t5-deepspeed#3-results--experiments) 10 个小时，我们需要 8 个 A100 40GB，成本约为 322 美元。

我们可以将模型保存下来以用于后面的推理和评估。我们暂时将其保存到磁盘，但你也可以使用 `model.push_to_hub` 方法将其上传到 [Hugging Face Hub](https://huggingface.co/docs/hub/main)。

In [10]:
# Save our LoRA model & tokenizer results
peft_model_id="results"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)
# if you want to save the base model to call
# trainer.model.base_model.save_pretrained(peft_model_id)

('results\\tokenizer_config.json',
 'results\\special_tokens_map.json',
 'results\\spiece.model',
 'results\\added_tokens.json',
 'results\\tokenizer.json')

最后生成的 LoRA checkpoint 文件很小，仅需 84MB 就包含了从 `samsum` 数据集上学到的所有知识。

## 4. 使用 LoRA FLAN-T5 进行评估和推理

我们将使用 `evaluate` 库来评估 `rogue` 分数。我们可以使用 `PEFT` 和 `transformers` 来对 FLAN-T5 XXL 模型进行推理。对 FLAN-T5 XXL 模型，我们至少需要 18GB 的​​ GPU 显存。

In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc. 
peft_model_id = "results"
config = PeftConfig.from_pretrained(peft_model_id)

# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path,  load_in_8bit=True,  device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model.eval()

print("Peft model loaded")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: d:\program files\python3.7.9\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary d:\program files\python3.7.9\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

d:\program files\python3.7.9\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\ProgramData\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Peft model loaded


我们用测试数据集中的一个随机样本来试试摘要效果。

In [2]:
from datasets import load_dataset 
from random import randrange


# Load dataset from the hub and get a sample
dataset = load_dataset("samsum")
sample = dataset['test'][randrange(len(dataset["test"]))]

input_ids = tokenizer(sample["dialogue"], return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=10, do_sample=True, top_p=0.9)
print(f"input sentence: {sample['dialogue']}\n{'---'* 20}")

print(f"summary:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")

Found cached dataset samsum (D:/ProgramData/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e)


  0%|          | 0/3 [00:00<?, ?it/s]

input sentence: Ana: You sleeping?
Catherine: Not yet.
Ana: Wanna go visit grandma tomorrow? I miss her.
Catherine: Yeah that would be nice :) I'll call you when I wake up
Ana: Oki :) sleep well, good night.
Catherine:  Good night, u too.
------------------------------------------------------------
summary:
Catherine will call Ana in the morning. Ana and


不错！我们的模型有效！现在，让我们仔细看看，并使用 `test` 集中的全部数据对其进行评估。为此，我们需要实现一些工具函数来帮助生成摘要并将其与相应的参考摘要组合到一起。评估摘要任务最常用的指标是 [rogue_score](https://en.wikipedia.org/wiki/ROUGE_(metric))，它的全称是 Recall-Oriented Understudy for Gisting Evaluation。与常用的准确率指标不同，它将生成的摘要与一组参考摘要进行比较。

In [3]:
import evaluate
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm

# Metric
metric = evaluate.load("rouge")

def evaluate_peft_model(sample,max_target_length=50):
    # generate summary
    outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(), do_sample=True, top_p=0.9, max_new_tokens=max_target_length)    
    prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
    # decode eval sample
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
    labels = tokenizer.decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    return prediction, labels

# load test dataset from distk
test_dataset = load_from_disk("data/eval/").with_format("torch")

# run predictions
# this can take ~45 minutes
predictions, references = [] , []
for sample in tqdm(test_dataset):
    p,l = evaluate_peft_model(sample)
    predictions.append(p)
    references.append(l)

# compute metric 
rogue = metric.compute(predictions=predictions, references=references, use_stemmer=True)

# print results 
print(f"Rogue1: {rogue['rouge1']* 100:2f}%")
print(f"rouge2: {rogue['rouge2']* 100:2f}%")
print(f"rougeL: {rogue['rougeL']* 100:2f}%")
print(f"rougeLsum: {rogue['rougeLsum']* 100:2f}%")

# Rogue1: 50.386161%
# rouge2: 24.842412%
# rougeL: 41.370130%
# rougeLsum: 41.394230%

100%|██████████| 819/819 [1:16:58<00:00,  5.64s/it]


Rogue1: 50.738340%
rouge2: 24.992193%
rougeL: 41.548080%
rougeLsum: 41.518933%


我们 PEFT 微调后的 ​​FLAN-T5-XXL 在测试集上取得了 `50.38%` 的 rogue1 分数。相比之下，[flan-t5-base 的全模型微调获得了 47.23 的 rouge1 分数](https://www.philschmid.de/fine-tune-flan-t5)。rouge1 分数提高了 `3%` 。

令人难以置信的是，我们的 LoRA checkpoint 只有 84MB，而且性能比对更小的模型进行全模型微调后的 checkpoint 更好。

> 英文原文: <url> https://www.philschmid.de/fine-tune-flan-t5-peft </url>

> 原文作者：Philipp Schmid

> 译者: Matrix Yao (姚伟峰)，英特尔深度学习工程师，工作方向为 transformer-family 模型在各模态数据上的应用及大规模模型的训练推理。